In [ ]:
import pandas as pd
import numpy as np
import json
import re
import seaborn as sns
import sys
import sqlite3

START_YEAR = 2010
END_YEAR = 2020
GROSS_PAY = 'Gross Pay'
REG_PAY = 'Regular Pay'

# id is just the row number PER YEAR, but not super meaningful.
# it is included in the JSON data.
COLUMNS = ['id', 'Year', 'Location', 'First Name', 'Last Name', 'Title', 'Gross Pay', 'Regular Pay', 'Overtime Pay', 'Other Pay']
# LECT_PREFIX = 'LECT-' # Caputes 'LECT-AY', excludes LECT PSOE / LECT SOE

def safe_parse(value):
    try:
        return float(value)
    except:
        return value

DATA_LOADED = {}
def load_data_for_year(year):
    raw_data = open(f"json/all-records-{year}.json").read()
    raw_data = raw_data.replace("'", '"')
    raw_data = re.sub(r'[\x00-\x1f\x7f-\x9f]', '', raw_data)
    parsed = json.loads(raw_data)
    data = [ [ safe_parse(value) for value in row['cell'] ] for row in parsed['rows'] ]

    df = pd.DataFrame(data, columns= COLUMNS)
    return df

In [ ]:
conn = sqlite3.connect('uc_salaries')
c = conn.cursor()

c.execute('''
          CREATE TABLE IF NOT EXISTS salaries (
            id INTEGER PRIMARY KEY,
            uc_id INTEGER,
            year INTEGER,
            location TEXT,
            first_name TEXT,
            last_name TEXT,
            title TEXT,
            gross_pay FLOAT,
            regular_pay FLOAT,
            overtime_pay FLOAT,
            other_pay FLOAT
          )
          ''')

In [ ]:
salaries_2020 = load_data_for_year('2020')
salaries_2020.head()

,id,Year,Location,First Name,Last Name,Title,Gross Pay,Regular Pay,Overtime Pay,Other Pay
0,1.0,2020.0,Los Angeles,,,TEACHER-UNEX-CONTRACT YR,3300.0,0.0,0.0,3300.0
1,2.0,2020.0,Riverside,,,LECT IN SUMMER SESSION,4792.0,0.0,0.0,4792.0
2,3.0,2020.0,San Diego,,,TEACHER-UNEX-CONTRACT YR,19415.0,0.0,0.0,19415.0
3,4.0,2020.0,San Diego,,,TEACHER-UNEX-CONTRACT YR,51397.0,0.0,0.0,51397.0
4,5.0,2020.0,San Diego,,,RSCH DATA ANL 3,63313.0,63313.0,0.0,0.0


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4848061e-35b7-4752-bb33-0d4be542a4b2' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>